# Predicting Airbnb Prices

 Internet marketplace for short-term home and appartment rentals
 <br>If hosts try to charge above market price --> renters will go away
 <br>If price too low --> Airbnb will miss out on potential revenue
 <br>--> optimal price

 <br>One strategy:
 <br>find few similar listing --> average listed price for most similar ones --> set listing price to calculated average price

 <br>--> build ML model with knn (k-nearest neighbors) to automate process

## Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

## Importing the dataset

In [2]:
# No data on listing from Airbnb directly but inside Airbnb group has extracted data on website sample

dc_listings = pd.read_csv('dc_airbnb.csv.gz')

print('{0} rows and {1} columns in this Dataframe.'.format(dc_listings.shape[0], dc_listings.shape[1]))

# Display Full Dataframe 
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(dc_listings.head())

3723 rows and 92 columns in this Dataframe.


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,7087327,https://www.airbnb.com/rooms/7087327,20151002231825,2015-10-03,Historic DC Condo-Walk to Capitol!,Professional pictures coming soon! Welcome to ...,NaN,Professional pictures coming soon! Welcome to ...,none,NaN,NaN,NaN,https://a2.muscache.com/ac/pictures/105507202/...,https://a2.muscache.com/im/pictures/105507202/...,https://a2.muscache.com/ac/pictures/105507202/...,https://a2.muscache.com/ac/pictures/105507202/...,15830506,https://www.airbnb.com/users/show/15830506,Lize & Greg,2014-05-21,"Washington, District of Columbia, United States","We are two fun, friendly entrepreneurs living ...",within a few hours,92%,91%,f,https://a1.muscache.com/ac/users/15830506/prof...,https://a1.muscache.com/ac/users/15830506/prof...,Truxton Circle,26,26,"['email', 'phone', 'facebook', 'google', 'revi...",t,t,"3rd Street Southeast, Washington, DC 20003, Un...",Capitol Hill,"Capitol Hill, Lincoln Park",NaN,Washington,DC,20003,D.C.,"Washington, DC",US,United States,38.890046,-77.002808,t,House,Entire home/apt,4,1.0,1.0,2.0,Real Bed,"{TV,""Wireless Internet"",""Air Conditioning"",Kit...",NaN,$160.00,"$1,125.00","$5,225.00",$100.00,$115.00,1,$0.00,1,1125,today,t,0,0,8,283,2015-10-02,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,flexible,f,f,18,NaN
1,975833,https://www.airbnb.com/rooms/975833,20151002231825,2015-10-03,Spacious Capitol Hill Townhouse,NaN,Beautifully renovated Capitol Hill townhouse. ...,Beautifully renovated Capitol Hill townhouse. ...,none,NaN,NaN,NaN,https://a2.muscache.com/ac/pictures/14613245/d...,https://a2.muscache.com/im/pictures/14613245/d...,https://a2.muscache.com/ac/pictures/14613245/d...,https://a2.muscache.com/ac/pictures/14613245/d...,5338703,https://www.airbnb.com/users/show/5338703,Sebastian,2013-03-05,"New York, New York, United States","Hi I'm Sebastian. \r\nAfter ten years in DC, m...",within a day,90%,100%,f,https://a1.muscache.com/ac/users/5338703/profi...,https://a1.muscache.com/ac/users/5338703/profi...,Capitol Hill,1,1,"['email', 'phone', 'reviews']",t,f,"11th Street Southeast, Washington, DC 20003, U...",Capitol Hill,"Capitol Hill, Lincoln Park",NaN,Washington,DC,20003,D.C.,"Washington, DC",US,United States,38.880413,-76.990485,t,House,Entire home/apt,6,3.0,3.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",NaN,$350.00,NaN,NaN,NaN,$100.00,1,$0.00,2,30,today,t,12,38,68,343,2015-10-02,65,2013-03-22,2015-09-28,94.0,10.0,9.0,10.0,10.0,9.0,9.0,f,NaN,"DISTRICT OF COLUMBIA, WASHINGTON",f,strict,f,f,1,2.11
2,8249488,https://www.airbnb.com/rooms/8249488,20151002231825,2015-10-03,Spacious/pri

In [3]:
# iterating the columns 
for col in dc_listings.columns: 
    print(col) 

id
listing_url
scrape_id
last_scraped
name
summary
space
description
experiences_offered
neighborhood_overview
notes
transit
thumbnail_url
medium_url
picture_url
xl_picture_url
host_id
host_url
host_name
host_since
host_location
host_about
host_response_time
host_response_rate
host_acceptance_rate
host_is_superhost
host_thumbnail_url
host_picture_url
host_neighbourhood
host_listings_count
host_total_listings_count
host_verifications
host_has_profile_pic
host_identity_verified
street
neighbourhood
neighbourhood_cleansed
neighbourhood_group_cleansed
city
state
zipcode
market
smart_location
country_code
country
latitude
longitude
is_location_exact
property_type
room_type
accommodates
bathrooms
bedrooms
beds
bed_type
amenities
square_feet
price
weekly_price
monthly_price
security_deposit
cleaning_fee
guests_included
extra_people
minimum_nights
maximum_nights
calendar_updated
has_availability
availability_30
availability_60
availability_90
availability_365
calendar_last_scraped
number_of_re

In [4]:
# dc_listings.columns.get_loc("price") get column index

dc_listings_cleaned = dc_listings.loc[:, ['accommodates', # the number of guests the rental can accommodate
                                          'bedrooms', # number of bedrooms included in the rental
                                          'bathrooms', # number of bathrooms included in the rental
                                          'beds', # number of beds included in the rental
                                          'minimum_nights', # minimum number of nights a guest can stay for the rental
                                          'maximum_nights', # maximum number of nights a guest can stay for the rental
                                          'number_of_reviews', # number of reviews that previous guests have left
                                          'price']] # nightly price for the rental

dc_listings_cleaned.head()

,accommodates,bedrooms,bathrooms,beds,minimum_nights,maximum_nights,number_of_reviews,price
0,4,1.0,1.0,2.0,1,1125,0,$160.00
1,6,3.0,3.0,3.0,2,30,65,$350.00
2,1,1.0,2.0,1.0,2,1125,1,$50.00
3,2,1.0,1.0,1.0,1,1125,0,$95.00
4,4,1.0,1.0,1.0,7,1125,0,$50.00


In [5]:
dc_listings_cleaned.to_excel('dc_airbnb_cleaned.xlsx', index = False) #save to excel file

## Building a Basic KNN Model

KNN algorithm --> compare listing to similar listings and take average price

1) select number of similar listings k to compare with.
<br>2) calculate how similar each listing is using a similarity metric
<br>3) rank each listing using similarity metric and select the first k listings.
<br>4) calculate mean price for k similar listings, and use that as list price.

  --> predict a continuous value like price
  <br>--> main similarity metric that’s used is Euclidean distance... 




$d = \sqrt{(q_1+p_1)^2+(q_2+p_2)^2+(q_3+p_3)^2+(q_4+p_4)^2+(q_5+p_5)^2+...+(q_n+p_n)^2}$

Let’s start by breaking things down and looking at just one column from data set. 
<br>Here’s the formula for just one feature: $d = \sqrt{(q_1+p_1)^2}$  --> $d = \left\lvert{q_1+p_1}\right\rvert$

--> absolute value of the difference between observation and data point to predict for the feature used

In [6]:
# Example : living space to rent can accommodate 3 people.

acc_value = 3

first_living_space_value = dc_listings_cleaned.loc[0,'accommodates'] # value of Dataframe's first row of accomodates column
first_distance = np.abs(first_living_space_value - acc_value)
print(first_distance)

1


In [7]:
# Euclidean distance for each observation in data set
# --> look at the range of values --> pd.value_counts()

dc_listings_cleaned['distance'] = np.abs(dc_listings_cleaned.accommodates - acc_value)
display(dc_listings_cleaned.sample(10))
print(dc_listings_cleaned.distance.value_counts().sort_index())

,accommodates,bedrooms,bathrooms,beds,minimum_nights,maximum_nights,number_of_reviews,price,distance
2697,4,1.0,1.0,1.0,2,30,0,$398.00,1
3637,4,1.0,1.0,2.0,3,180,0,$125.00,1
3566,2,1.0,1.0,1.0,2,1125,1,$115.00,1
2557,4,2.0,1.0,2.0,1,1125,0,$95.00,1
462,6,3.0,2.0,3.0,2,1125,11,$495.00,3
2709,2,1.0,1.0,1.0,1,1125,0,$75.00,1
3324,2,1.0,1.0,1.0,3,14,5,$80.00,1
1896,6,2.0,1.0,3.0,3,1125,10,$119.00,3
3070,2,1.0,1.0,1.0,5,30,11,$129.00,1
2702,4,2.0,1.0,2.0,1,1125,0,$100.00,1


0      461
1     2294
2      503
3      279
4       35
5       73
6       17
7       22
8        7
9       12
10       2
11       4
12       6
13       8
Name: distance, dtype: int64


In [8]:
# 461 listings that have a distance of 0 --> accommodate the same number of people as listing --> good place to start

dc_listings_cleaned = dc_listings_cleaned.sample(frac=1) # randomize the ordering of 100% of the observations
dc_listings_cleaned = dc_listings_cleaned.sort_values('distance') # --> select the first five rows with a distance of 0
print(dc_listings_cleaned.price.head())

747     $175.00
1456     $99.00
1791     $95.00
2148    $125.00
48       $55.00
Name: price, dtype: object


In [9]:
print(dc_listings_cleaned.dtypes)

accommodates           int64
bedrooms             float64
bathrooms            float64
beds                 float64
minimum_nights         int64
maximum_nights         int64
number_of_reviews      int64
price                 object
distance               int64
dtype: object


In [10]:
# price = object because of $ and ,
# remove $  and , --> convert price from object to float

dc_listings_cleaned['price'] = dc_listings_cleaned['price'].str.replace("\$|,",'').astype(float)

In [11]:
print(dc_listings_cleaned.dtypes)

accommodates           int64
bedrooms             float64
bathrooms            float64
beds                 float64
minimum_nights         int64
maximum_nights         int64
number_of_reviews      int64
price                float64
distance               int64
dtype: object


In [12]:
dc_listings_cleaned.rename(columns = {'price':'price ($)'}, inplace = True)

display(dc_listings_cleaned.sample(10))

,accommodates,bedrooms,bathrooms,beds,minimum_nights,maximum_nights,number_of_reviews,price ($),distance
428,5,3.0,2.0,3.0,1,1125,0,800.0,2
68,2,1.0,1.0,1.0,1,365,0,60.0,1
2078,2,1.0,1.0,1.0,2,30,318,75.0,1
1506,2,1.0,2.0,1.0,10,365,46,85.0,1
3403,2,0.0,1.0,1.0,1,1125,0,100.0,1
1205,3,1.0,1.0,1.0,2,1125,6,150.0,0
2365,4,1.0,1.0,1.0,3,1125,1,119.0,1
1999,1,1.0,1.0,1.0,2,60,21,100.0,2
96,5,2.0,2.0,2.0,3,365,34,299.0,2
3243,2,1.0,1.5,1.0,2,1125,1,45.0,1


In [13]:
mean_price = dc_listings_cleaned['price ($)'].iloc[:5].mean()

print('An appartment that can accomodate 3 persons should be listed for ${0} a night.'.format(mean_price))

An appartment that can accomodate 3 persons should be listed for $109.8 a night.


good start but to know model accuracy ---> impossible to optimize and improve...

## Evaluating model

### Splitting the dataset into the Training set and Test set

split the 3,723 rows of our data set into two parts: train_df and test_df in a 75%-25% split.

In [14]:
train_set_count = int(round(75/100 * dc_listings_cleaned.shape[0], 0))
test_set_count = int(round(25/100 * dc_listings_cleaned.shape[0], 0)) # more accurate result with round()

print('{0} rows in training set and {1} rows in test set.'.format(train_set_count, test_set_count))

2792 rows in training set and 931 rows in test set.


In [15]:
# Reminder: rows already randomnised

dc_listings_cleaned.drop('distance',axis=1)
train_df = dc_listings_cleaned.copy().iloc[:train_set_count] # up to row
test_df = dc_listings_cleaned.copy().iloc[train_set_count:] # from row to the end

### Fitting classifier to the training set

In [16]:
# model --> function

def predict_price(new_listing_value,feature_column):
    temp_df = train_df
    temp_df['distance'] = np.abs(dc_listings_cleaned[feature_column] - new_listing_value) # Euclidean distance
    temp_df = temp_df.sort_values('distance') # sort by shortest to longest distance
    knn_5 = temp_df['price ($)'].iloc[:5] # 5 nearest neighbours = 5 first rows where distance = 0
    predicted_price = knn_5.mean() # get average price
    return(predicted_price)

In [17]:
# use this function to predict values for test dataset using the accommodates column

test_df['predicted_price'] = test_df['accommodates'].apply(predict_price,feature_column='accommodates')

### RMSE (root mean squared error) to evaluate model

For many prediction tasks --> penalize predicted values that are further away from the actual value much more than those that are closer to the actual value.
<br> --> mean of the squared error values, which is called the root mean squared error (RMSE). Here’s the formula for RMSE:

$RMSE = \sqrt{\frac{(actual_1-predicted_1)^2+(actual_2-predicted_2)^2+(actual_3-predicted_3)^2+...+(actual_n-predicted_n)^2}{n}}$

<br>Taking the difference between each predicted value and the actual value (or error),
<br>Squaring this difference (square),
<br>Taking the mean of all the squared differences (mean), and
<br>Taking the square root of that mean (root).

In [18]:
test_df['squared_error'] = (test_df['predicted_price'] - test_df['price ($)'])**(2)
mse = test_df['squared_error'].mean()
rmse = mse ** (1/2)


print('Root mean squared error of approx. ${0}.'.format(int(rmse)))

Root mean squared error of approx. $241.


pretty big scale —-> still fairly far from accurate prediction

## Comparing different models

--> create some predictions using different columns and look at how our error varies

In [19]:
for feature in ['accommodates','bedrooms','bathrooms','number_of_reviews']:
    test_df['predicted_price'] = test_df['accommodates'].apply(predict_price,feature_column=feature)
    test_df['squared_error'] = (test_df['predicted_price'] - test_df['price ($)'])**(2)
    mse = test_df['squared_error'].mean()
    rmse = mse ** (1/2)
    print("RMSE for the {} column: ${}.".format(feature, int(rmse)))

RMSE for the accommodates column: $241.
RMSE for the bedrooms column: $212.
RMSE for the bathrooms column: $215.
RMSE for the number_of_reviews column: $244.


error rates are quite high relative to the range of prices of the listing in data set. 
<br>Predictions with these sorts of error rates would not be very helpful.

univariate model (model with only one feature) --> multivariate model

### Data Cleaning

In [20]:
# drop distance column
dc_listings_cleaned = dc_listings_cleaned.drop('distance',axis=1)
# drop rows with missing values
dc_listings_cleaned = dc_listings_cleaned.dropna(how = 'any')

In [21]:
display(dc_listings_cleaned.head())

print('{0} rows and {1} columns in this Dataframe.'.format(dc_listings_cleaned.shape[0], dc_listings_cleaned.shape[1]))

,accommodates,bedrooms,bathrooms,beds,minimum_nights,maximum_nights,number_of_reviews,price ($)
747,3,1.0,1.0,2.0,1,1125,2,175.0
1456,3,1.0,1.0,1.0,2,1125,15,99.0
1791,3,1.0,1.0,1.0,2,1125,16,95.0
2148,3,1.0,1.0,1.0,2,1125,4,125.0
48,3,1.0,1.0,2.0,1,1125,5,55.0


3671 rows and 8 columns in this Dataframe.


### Feature scaling

In [22]:
X = dc_listings_cleaned.iloc[:, :7].values
y = dc_listings_cleaned.iloc[:, -1].values

In [23]:
from sklearn.model_selection import train_test_split

# Training Dataset: The sample of data used to fit the model.
# The actual dataset that we use to train the model (weights and biases in the case of Neural Network). 
# The model sees and learns from this data.

# Test Dataset: The sample of data used to provide an unbiased evaluation of a final model fit on the training dataset.
# provides the gold standard used to evaluate the model.
# only used once a model is completely trained
# contains carefully sampled data that spans the various classes that the model would face, when used in the real world.

# Split dataset into training set (X_train, y_train) and test set (X_test, y_test), 
# test_size = proportion of the dataset to include in the test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [24]:
from sklearn.preprocessing import StandardScaler

# K-nearest neighbors requires features to be normalized as important to have values on same scale for acurate results
# Obviously age and estimated salary features  have different units, one in years, the other in $
# Standardization involves rescaling the features such that 
# -->they have the properties of a standard normal distribution with a mean of zero and a standard deviation of one
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [25]:
X_train_df = pd.DataFrame(X_train) 
X_test_df = pd.DataFrame(X_test)
y_train_df = pd.DataFrame(y_train) 
y_test_df = pd.DataFrame(y_test)

X_train_df['y'] = y_train_df
train_set = X_train_df

X_test_df['y'] = y_test_df
test_set = X_test_df

train_set.columns = dc_listings_cleaned.columns
test_set.columns = dc_listings_cleaned.columns

display(train_set)
display(test_set)

,accommodates,bedrooms,bathrooms,beds,minimum_nights,maximum_nights,number_of_reviews,price ($)
0,-0.599048,-1.446235,-0.439306,-0.563326,-0.559760,-0.019143,-0.311980,99.0
1,-1.100319,-0.246993,-0.439306,-0.563326,0.851430,-0.019143,-0.510688,40.0
2,-0.599048,-0.246993,-0.439306,-0.563326,-0.089363,-0.019143,-0.510688,135.0
3,-0.599048,-0.246993,-0.439306,-0.563326,0.851430,-0.019153,-0.444452,209.0
4,-0.097778,-0.246993,-0.439306,-0.563326,0.851430,-0.019143,-0.510688,289.0
...,...,...,...,...,...,...,...,...
2748,-0.599048,-1.446235,-0.439306,-0.563326,-0.559760,-0.019143,-0.444452,82.0
2749,-0.097778,0.952250,1.256919,0.297772,1.321827,-0.019143,-0.444452,159.0
2750,1.406034,0.952250,-0.439306,2.019968,-0.559760,-0.019143,-0.510688,189.0
2751,-0.599048,-0.246993,0.408807,-0.563326,-0.559760,-0.019170,1.112092,85.0


,accommodates,bedrooms,bathrooms,beds,minimum_nights,maximum_nights,number_of_reviews,price ($)
0,-0.599048,-0.246993,-0.439306,-0.563326,-0.559760,-0.019143,-0.510688,90.0
1,-0.599048,-0.246993,-0.439306,-0.563326,-0.089363,-0.019171,-0.510688,190.0
2,-0.599048,-0.246993,-0.439306,-0.563326,0.851430,-0.019170,-0.311980,139.0
3,1.406034,0.952250,1.256919,0.297772,-0.089363,-0.019143,-0.411334,154.0
4,-0.599048,-0.246993,-0.439306,-0.563326,-0.559760,-0.019143,-0.378216,70.0
...,...,...,...,...,...,...,...,...
913,-0.599048,-0.246993,-0.439306,-0.563326,1.321827,-0.019143,-0.477570,95.0
914,-0.599048,-0.246993,-0.439306,-0.563326,-0.559760,-0.019171,-0.146390,99.0
915,1.406034,2.151492,1.256919,2.019968,-0.089363,-0.019143,-0.444452,190.0
916,0.403493,0.952250,-0.439306,1.158870,-0.089363,-0.019167,0.251025,99.0


In [26]:
# Find distance between 1st and 5th row to practice

from scipy.spatial import distance
first_listing = test_set.iloc[0][['accommodates', 'bathrooms']]
fifth_listing = test_set.iloc[20][['accommodates', 'bathrooms']]
first_fifth_distance = distance.euclidean(first_listing, fifth_listing)
round(first_fifth_distance, 2)

0.5

## Creating a multivariate KNN model

In [27]:
# cdist allows to pass multiple rows at once

def predict_price_multivariate(new_listing_value,feature_columns):
    temp_df = train_set
    temp_df['distance'] = distance.cdist(temp_df[feature_columns],[new_listing_value[feature_columns]])
    temp_df = temp_df.sort_values('distance')
    knn_5 = temp_df['price ($)'].iloc[:5]
    predicted_price = knn_5.mean()
    return(predicted_price)

cols = ['accommodates', 'bathrooms']
test_set['predicted_price'] = test_set[cols].apply(predict_price_multivariate,feature_columns=cols,axis=1)
test_set['squared_error'] = (test_set['predicted_price'] - test_set['price ($)'])**(2)
mse = test_set['squared_error'].mean()
rmse = mse ** (1/2)
print('Root mean squared error of approx. ${0}.'.format(int(rmse)))

Root mean squared error of approx. $148.


significant RMSE improvement because use of 2 features rather than just one

scikit-learn workflow consists of four main steps:

<br>Instantiate the specific machine learning model you want to use.
<br>Fit the model to the training data.
<br>Use the model to make predictions.
<br>Evaluate the accuracy of the predictions.

In [28]:
from sklearn.neighbors import KNeighborsRegressor
# Algorithm used to compute the nearest neighbors: will use a brute-force search
knn = KNeighborsRegressor(algorithm='brute')

## Fitting a Model and Making Predictions

Now, we can fit the model to the data using the fit method. For all models, the fit method takes in two required parameters:

a matrix-like object, containing the feature columns we want to use from the training set.

a list-like object, containing correct target values.

In [29]:
knn.fit(train_set[cols], train_set['price ($)'])

KNeighborsRegressor(algorithm='brute', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [33]:
b = knn.fit(train_set[cols], train_set['price ($)'])
X_train_df = pd.DataFrame(X_train) 

KNeighborsRegressor(algorithm='brute', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

Now that we’ve specified the training data we want used to make predictions, we can use the predict method to make predictions about the test set. The predict method has only one required parameter:

A matrix-like object, containing the feature columns from the data set we want to make predictions about

In [30]:
two_features_predictions = knn.predict(test_set[cols])

In [31]:
from sklearn.metrics import mean_squared_error
two_features_mse = mean_squared_error(test_set['price ($)'], two_features_predictions)
two_features_rmse = two_features_mse ** (1/2)
print('Root mean squared error of approx. ${0}.'.format(int(two_features_rmse)))

Root mean squared error of approx. $147.


## Using More Features

In [32]:
knn = KNeighborsRegressor(algorithm='brute')
cols = ['accommodates','bedrooms','bathrooms','beds']
knn.fit(train_set[cols], train_set['price ($)'])
four_features_predictions = knn.predict(test_set[cols])
four_features_mse = mean_squared_error(test_set['price ($)'], four_features_predictions)
four_features_rmse = four_features_mse ** (1/2)
print('Root mean squared error of approx. ${0}.'.format(int(four_features_rmse)))

Root mean squared error of approx. $153.


--> adding more features does not necessarily produce a more accurate model.